In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.sparse import csr_matrix
import sklearn
import itertools
import scipy as sp
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
data = pd.read_csv("movie_recommendation.csv")
movies = pd.read_csv("movies2.csv")

In [70]:
data

,userId,movieId,rating,date,title,genres,link,published year
0,1.0,1.0,4.0,30-07-00,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
1,5.0,1.0,4.0,08-11-96,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
2,7.0,1.0,4.5,25-01-05,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
3,15.0,1.0,2.5,13-11-17,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
4,17.0,1.0,4.5,18-05-11,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
...,...,...,...,...,...,...,...,...
96644,610.0,160341.0,2.5,19-11-16,Bloodmoon (1997),Action|Thriller,www.imdb.com/title/tt118745,(1997)
96645,610.0,160527.0,4.5,19-11-16,Sympathy for the Underdog (1971),Action|Crime|Drama,www.imdb.com/title/tt66806,(1971)
96646,610.0,160836.0,3.0,03-05-17,Hazard (2005),Action|Drama|Thriller,www.imdb.com/title/tt798722,(2005)
96647,610.0,163937.0,3.5,04-05-17,Blair Witch (2016),Horror|Thriller,www.imdb.com/title/tt1540011,(2016)


## Naiev collaborative filltering

In [71]:
movie_user_mat = data.pivot(index='movieId', columns='userId', values='rating').fillna(0)

In [72]:
movie_user_mat

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,601.0,602.0,603.0,604.0,605.0,606.0,607.0,608.0,609.0,610.0
movieId,,,,,,,,,,,,,,,,,,,,,
1.0,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [74]:
movie_user_mat_sparse

<9472x610 sparse matrix of type '<class 'numpy.float64'>'
	with 96649 stored elements in Compressed Sparse Row format>

In [75]:
from sklearn.neighbors import NearestNeighbors
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [76]:
df_movies = movies[["movieId","title"]]

In [77]:
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(df_movies.set_index('movieId').loc[movie_user_mat.index].title))
}



In [78]:
from fuzzywuzzy import fuzz

def fuzzy_matching(mapper, fav_movie, verbose=True):
    """
    return the closest match via fuzzy ratio. 
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data
    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True
    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]

In [79]:
def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    """
    return top n similar movie recommendations based on user's input movie
    Parameters
    ----------
    model_knn: sklearn model, knn model
    data: movie-user matrix
    mapper: dict, map movie title name to index of the movie in data
    fav_movie: str, name of user input movie
    n_recommendations: int, top n recommendations
    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
    model_knn.fit(data)
    
    # get input movie index
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    
    raw_recommends = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    test_array = []
    for i, (idx, dist) in enumerate(raw_recommends):
        test_array.append(reverse_mapper[idx])
    
    return test_array

In [80]:
test_array = []

def test(my_favorite):
        test_array.append(make_recommendation(
        model_knn=model_knn,
        data=movie_user_mat_sparse,
        fav_movie=my_favorite,
        mapper=movie_to_idx,
        n_recommendations=10))
    
my_favorite = ["toy story"]
for movie in my_favorite:
    test(movie)

Found possible matches in our database: ['Toy Story (1995)', 'Toy Story 3 (2010)', 'Toy Story 2 (1999)']

Recommendations for toy story:


In [81]:
test_array

[['Back to the Future (1985)',
  'Groundhog Day (1993)',
  'Mission: Impossible (1996)',
  'Lion King, The (1994)',
  'Star Wars: Episode VI - Return of the Jedi (1983)',
  'Forrest Gump (1994)',
  'Star Wars: Episode IV - A New Hope (1977)',
  'Independence Day (a.k.a. ID4) (1996)',
  'Jurassic Park (1993)',
  'Toy Story 2 (1999)']]

## -----------------------------------------------------------------------------------------------

## ----------------------------------------------------------------------------------------------

In [82]:
data2 = data[["userId", "movieId", "rating"]]
group1 = data2.groupby(['userId'],as_index = False, sort=False)["rating"].sum()
data2 = pd.merge(data2, group1, on = "userId", how = "left", sort = False)
group2 = data2.groupby(['userId'],as_index = False, sort=False)["rating_x"].count()
data2 = pd.merge(data2, group2, on = "userId", how = "left", sort = False)
data2["normalize_rating"] = (data2["rating_x_x"] - data2["rating_y"]) / data2["rating_x_y"]
data_normalize = data2.rename(columns = {"rating_x_x":"rating", "rating_y":"sum", "rating_x_y":"count"})

In [83]:
user_similarity_mat = data_normalize.pivot(index='movieId', columns='userId', values='normalize_rating').fillna(0)
user_similarity_mat.reset_index(inplace=True)
user_similarity_mat = user_similarity_mat.drop('movieId', axis=1)
user_ratings = pd.DataFrame(user_similarity_mat, index = user_similarity_mat.columns, columns = user_similarity_mat.columns)
similarity = cosine_similarity(user_ratings)
user_ratings = pd.DataFrame(similarity, index = user_ratings.index, columns = user_ratings.index)

#piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
#user_similarity = 1 - spatial.distance.cosine(piv_sparse)
#user_similarity = cosine_similarity(piv_sparse)


for i in range(len(user_ratings.columns)) :
    for j in range(len(user_ratings.columns)) :
        user_ratings.iloc[i,j] = 1-cosine(user_similarity_mat.iloc[:,i],user_similarity_mat.iloc[:,j])

In [90]:
user_ratings

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,601.0,602.0,603.0,604.0,605.0,606.0,607.0,608.0,609.0,610.0
userId,,,,,,,,,,,,,,,,,,,,,
1.0,1.000000,0.296749,0.102113,0.293768,0.314169,0.217245,0.159211,0.052030,0.424438,0.321074,...,0.237530,0.179380,0.042590,0.205315,0.000000,0.099750,0.094053,0.293002,0.0,0.461428
2.0,0.296749,1.000000,0.056821,0.391532,0.291379,0.389498,0.234639,0.285807,0.230094,0.234991,...,0.259767,0.408073,0.063824,0.252912,0.055684,0.000000,0.140795,0.192905,0.0,0.371397
3.0,0.102113,0.056821,1.000000,0.165346,0.081455,0.266181,0.152381,0.000000,0.102834,0.140527,...,0.000000,0.000000,0.000000,0.089950,0.000000,0.000000,0.403571,0.031541,0.0,0.050519
4.0,0.293768,0.391532,0.165346,1.000000,0.327143,0.483155,0.234873,0.350648,0.242747,0.271483,...,0.133982,0.381538,0.000000,0.262868,0.000000,0.000000,0.140530,0.136777,0.0,0.323699
5.0,0.314169,0.291379,0.081455,0.327143,1.000000,0.276292,0.165698,0.215349,0.394779,0.285038,...,0.225470,0.289577,0.043590,0.144722,0.086182,0.102109,0.096293,0.431290,0.0,0.450092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606.0,0.099750,0.000000,0.000000,0.000000,0.102109,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.101535,0.0,0.073100
607.0,0.094053,0.140795,0.403571,0.140530,0.096293,0.138522,0.377581,0.000000,0.085749,0.117233,...,0.000000,0.000000,0.000000,0.222885,0.000000,0.000000,1.000000,0.000000,0.0,0.068873
608.0,0.293002,0.192905,0.031541,0.136777,0.431290,0.148513,0.097526,0.097012,0.254694,0.215975,...,0.106141,0.191832,0.125502,0.108406,0.085728,0.101535,0.000000,1.000000,0.0,0.378927


def sim_movies(users, user_movie):
    movies = []
    for user in users:
        movies.append(data[data["userId"] == user[0]]["title"])
    movies = list(itertools.chain.from_iterable(movies))
    movies = list(set(movies))
    movies = set(movies) - set(user_movie)
    return movies

In [91]:
def top_users(user):
    users = []
    #if user not in user_sim_df.columns:
      #  return('No data available on user {}'.format(user))
    
    sim_values = user_ratings.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_ratings.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_users, sim_values,)
    for user, sim in zipped:
        users.append(user)
    #movies = sim_movies(users, user_movie)
    return users

In [92]:
sim_user = top_users(1)
#sim_user.append(1.0)
sim_user_data = pd.DataFrame()
sim_user.append(1)

for user in sim_user:
    sim_user_data = sim_user_data.append(data[data["userId"] == user])

### -----------------------------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------------------------
## movie based

In [93]:
sim_user_data = sim_user_data.drop_duplicates()

In [94]:
movie_similarity_mat = sim_user_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)
movie_similarity_mat

movieId,1.0,2.0,3.0,5.0,6.0,7.0,8.0,10.0,11.0,15.0,...,179819.0,180031.0,180045.0,180095.0,180497.0,180777.0,180985.0,184791.0,185135.0,187595.0
userId,,,,,,,,,,,,,,,,,,,,,
1.0,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18.0,3.5,3.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,4.5,0.0,0.0,4.5,0.0
275.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
321.0,0.0,5.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
324.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
414.0,4.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,0.0,4.0,0.0,3.5,2.5,0.0,3.5
432.0,2.5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
479.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
df = movie_similarity_mat.replace(0, np.NaN)
df = df.mean()

In [96]:
for column in movie_similarity_mat.columns:
    movie_similarity_mat[column][1] = df[column]

In [97]:
movie_similarity_mat.reset_index(inplace=True)
movie_similarity_mat = movie_similarity_mat.drop('userId', axis=1)

In [98]:
movie_ratings = pd.DataFrame(movie_similarity_mat, index = movie_similarity_mat.columns, columns = movie_similarity_mat.columns)

In [99]:
movie_ratings2 = movie_ratings.fillna(0)

In [100]:
similarity = cosine_similarity(movie_ratings2)

for i in range(len(movie_ratings.columns)) :
    for j in range(len(movie_ratings.columns)) :
        movie_ratings.iloc[i,j] = 1-cosine(movie_similarity_mat.iloc[:,i],movie_similarity_mat.iloc[:,j])

In [101]:
movie_ratings2 = pd.DataFrame(similarity, index = movie_ratings2.index, columns = movie_ratings2.index)

In [102]:
movie_ratings2

movieId,1.0,2.0,3.0,5.0,6.0,7.0,8.0,10.0,11.0,15.0,...,179819.0,180031.0,180045.0,180095.0,180497.0,180777.0,180985.0,184791.0,185135.0,187595.0
movieId,,,,,,,,,,,,,,,,,,,,,
1.0,1.000000,0.184222,0.109459,0.359997,0.312487,0.069578,0.101108,0.065884,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.184222,1.000000,0.037736,0.312903,0.131220,0.177411,0.180010,0.061560,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.109459,0.037736,1.000000,0.110712,0.142464,0.059860,0.095752,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.359997,0.312903,0.110712,1.000000,0.261275,0.189740,0.166741,0.072852,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6.0,0.312487,0.131220,0.142464,0.261275,1.000000,0.068122,0.089359,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180777.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
180985.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184791.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
def top_movies(movie):
    movies = []
    #if user not in user_sim_df.columns:
      #  return('No data available on user {}'.format(user))
    
    sim_values = movie_ratings.sort_values(by = movie, ascending=False).loc[:,movie].tolist()[1:20]
    sim_users = movie_ratings.sort_values(by = movie, ascending=False).index[1:20]
    zipped = zip(sim_users, sim_values,)
    for movie, sim in zipped:
        movies.append(movie)
    #movies = sim_movies(users, user_movie)
    return movies

In [104]:
watched_movies = data[data["userId"] == 1]

In [105]:
watched_movie_ids = watched_movies["movieId"]

In [106]:
sim_movies = []
for Id in watched_movie_ids:
    sim_movies.append(top_movies(Id))

In [107]:
sim_movies = list(itertools.chain.from_iterable(sim_movies))

In [108]:
for movie in sim_movies:
    print(df_movies[df_movies["movieId"] == movie]["title"].tolist())

['Father of the Bride Part II (1995)']
['Tom and Huck (1995)']
['Toy Story (1995)']
['Heat (1995)']
['Grumpier Old Men (1995)']
['Sabrina (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Grumpier Old Men (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Father of the Bride Part II (1995)']
['Jumanji (1995)']
['Gr

['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Toy Story (1995)']
['Grumpier Old Men (1995)']
['Jumanji (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Tom and Huck (1995)']
['Toy Story (1995)']
['Sabrina (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
[

['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Sabrina (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']

['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Toy Story (1995)']
['Heat (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['Gol

['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
[

['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Father of the Bride Part II (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Tom and Huck (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['Sabrina (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Toy Story

['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Father of the Bride Part II (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms 

['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Tom and Huck (1995)']
['Jumanji (1995)']
['Father of the Bride Part II (1995)']
['Toy Story (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Jumanji (1995)']
['Toy Story (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)'

['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Heat (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995)']
['Copycat (1995)']
['Assassins (1995)']
['Powder (1995)']
['Leaving Las Vegas (1995)']
['Now and Then (1995)']
['Toy Story (1995)']
['Jumanji (1995)']
['Grumpier Old Men (1995)']
['Heat (1995)']
['Sabrina (1995)']
['Tom and Huck (1995)']
['GoldenEye (1995)']
['American President, The (1995)']
['Cutthroat Island (1995)']
['Casino (1995)']
['Sense and Sensibility (1995)']
['Four Rooms (1995)']
['Ace Ventura: When Nature Calls (1995)']
['Get Shorty (1995

In [109]:
intersect_movies = list(set(watched_movies["movieId"]).intersection(sim_movies))

In [110]:
y_pred = []
y_test = []

for movie in intersect_movies:
    y_pred.append(df[movie])
    y_test.append(watched_movies[watched_movies["movieId"] == movie]["rating"].tolist())

In [111]:
y_pred

[3.8333333333333335, 3.6666666666666665, 3.6666666666666665]

In [112]:
y_test

[[4.0], [4.0], [4.0]]

In [113]:
import math
import sklearn.metrics as metrics
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

0.288675134594813


train_data = data["title"]
train_data = list(dict.fromkeys(train_data))